In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mooccubex-entities/comment.json
/kaggle/input/mooccubex-entities/course.json
/kaggle/input/mooccubex-entities/paper.json
/kaggle/input/mooccubex-entities/reply.json
/kaggle/input/mooccubex-entities/school.json
/kaggle/input/mooccubex-entities/video.json
/kaggle/input/mooccubex-entities/other.json
/kaggle/input/mooccubex-entities/problem.json
/kaggle/input/mooccubex-entities/user.json
/kaggle/input/mooccubex-entities/concept.json
/kaggle/input/mooccubex-entities/teacher.json
/kaggle/input/preprocessing-user-json/__results__.html
/kaggle/input/preprocessing-user-json/user-5-100.csv
/kaggle/input/preprocessing-user-json/__notebook__.ipynb
/kaggle/input/preprocessing-user-json/__output__.json
/kaggle/input/preprocessing-user-json/user-course.csv
/kaggle/input/preprocessing-user-json/custom.css


In [2]:
def pl_read_file(file_path):
    file_extension = os.path.splitext(file_path)[1]
    if file_extension == '.json':
        return pl.read_ndjson(file_path)
    elif file_extension == '.csv':
        return pl.read_csv(file_path)
    elif file_extension == '.txt':
        return pl.read_csv(file_path,separator='\t',has_header=False)
    else:
        raise ValueError(f"Unsupported file extension: {file_extension}")

In [3]:
course_df = pl_read_file("/kaggle/input/mooccubex-entities/course.json")

In [4]:
course_df.head()

id,name,field,prerequisites,about,resource
str,str,list[str],str,str,list[struct[3]]
"""C_584313""","""《资治通鉴》导读""","[""历史学"", ""中国语言文学""]","""""","""通过老师导读，同学们可深入这一经典文本内部，得以纵览千年历史…","[{[""第一课 导论与三家分晋"", ""导论"", ""导论""],""V_849"",""1.1.1""}, {[""第一课 导论与三家分晋"", ""智伯的覆亡"", ""智伯的覆亡""],""V_850"",""1.2.1""}, … {[""第十五课 隋唐霸业"", null, ""第十五课 隋唐霸业--习题""],""Ex_957"",""15.8""}]"
"""C_584329""","""微积分——极限理论与一元函数""","[""应用经济学"", ""数学"", … ""理论经济学""]","""""","""本课程是理工科的一门数学基础课，系统、全面地介绍了一元函数微…","[{[""序言"", ""序言"", ""序言""],""V_1350"",""1.1.1""}, {[""第一章 实数与函数"", ""第一节 实数集的界与确界"", ""实数集的界""],""V_1351"",""2.1.1""}, … {[""第八章 级数"", null, ""第八章 级数--第六节思考与练习""],""Ex_1545"",""9.9""}]"
"""C_584381""","""新闻摄影""","[""艺术学"", ""新闻传播学""]","""""","""掌握基本的摄影技能，了解图片新闻的工作方式，训练对生活的观察…","[{[""第一章 绪论"", ""第一讲 引言1"", ""引言1""],""V_1800"",""1.1.1""}, {[""第一章 绪论"", ""第二讲 引言2"", ""引言2""],""V_1801"",""1.2.1""}, … {[""大作业提交"", null, ""《大作业》提交--小节""],""Ex_1926"",""20.4""}]"
"""C_597208""","""数据挖掘：理论与算法""","[""计算机科学与技术""]","""""","""最有趣的理论+最有用的算法=不得不学的数据科学。""","[{[""走进数据科学：博大精深，美不胜收"", ""整装待发"", ""Video""],""V_2961"",""1.1.1""}, {[""走进数据科学：博大精深，美不胜收"", ""学而不思则罔"", ""Video""],""V_2962"",""1.3.1""}, … {[""美丽数据说：阆苑仙葩，美玉无瑕"", null, ""第十一章第一节测试题""],""Ex_3104"",""11.1""}]"
"""C_597225""","""大学计算机""",[],"""""","""大学计算机课程将以计算思维为导向，以计算机原理、概念为基础，…","[{[""第1周： 基于计算机的问题求解"", ""课程介绍"", ""开篇""],""V_4596"",""1.1.1""}, {[""第1周： 基于计算机的问题求解"", ""1.0 本章导学"", ""1.0 本章导学""],""V_4597"",""1.2.1""}, … {[""第9周：算法与程序设计"", null, ""第九周测验""],""Ex_4827"",""10.12""}]"


In [5]:
course_df.describe()

statistic,id,name,field,prerequisites,about,resource
str,str,str,f64,str,str,f64
"""count""","""3781""","""3781""",3781.0,"""3779""","""3779""",3781.0
"""null_count""","""0""","""0""",0.0,"""2""","""2""",0.0
"""mean""",null,null,null,null,null,null
"""std""",null,null,null,null,null,null
"""min""","""C_1017355""",""" Food Chemistry """,null,"""""","""""",null
"""25%""",null,null,null,null,null,null
"""50%""",null,null,null,null,null,null
"""75%""",null,null,null,null,null,null
"""max""","""C_956450""","""（疾风计划）面向对象程序设计（C++）""",null,"""高级语言程序设计、数据结构""","""（1）特色：课程资源建设，充分体现优质资源的共享。老师从繁杂…",null


In [6]:
course_resource = course_df.select(pl.col(['id','resource'])).explode('resource')

In [7]:
course_resource

id,resource
str,struct[3]
"""C_584313""","{[""第一课 导论与三家分晋"", ""导论"", ""导论""],""V_849"",""1.1.1""}"
"""C_584313""","{[""第一课 导论与三家分晋"", ""智伯的覆亡"", ""智伯的覆亡""],""V_850"",""1.2.1""}"
"""C_584313""","{[""第一课 导论与三家分晋"", ""智伯悲剧的反思"", ""智伯的覆亡讨论""],""V_851"",""1.3.1""}"
"""C_584313""","{[""第一课 导论与三家分晋"", null, ""第一课 导论与三家分晋--习题""],""Ex_856"",""1.4""}"
"""C_584313""","{[""第二课 战国前期的政治"", ""魏文侯治国"", ""魏文侯治国""],""V_857"",""2.1.1""}"
…,…
"""C_2329163""","{[""第四章 红色之旅实践写作"", ""4.16 求职简历"", ""4.16 求职简历""],""V_8630134"",""3.15""}"
"""C_2329163""","{[""第四章 红色之旅实践写作"", ""4.16 求职简历"", ""求职简历作业""],""Ex_8638642"",""3.15.1""}"
"""C_2329163""","{[""第四章 红色之旅实践写作"", ""4.17 实践—演讲稿"", ""4.17 实践—演讲稿""],""V_8630135"",""3.16""}"


In [8]:
course_resource=course_resource.unnest("resource")

In [9]:
course_video = course_resource.filter(pl.col("resource_id").str.starts_with("V"))
course_exercise = course_resource.filter(pl.col("resource_id").str.starts_with("Ex"))

In [10]:
course_video

id,titles,resource_id,chapter
str,list[str],str,str
"""C_584313""","[""第一课 导论与三家分晋"", ""导论"", ""导论""]","""V_849""","""1.1.1"""
"""C_584313""","[""第一课 导论与三家分晋"", ""智伯的覆亡"", ""智伯的覆亡""]","""V_850""","""1.2.1"""
"""C_584313""","[""第一课 导论与三家分晋"", ""智伯悲剧的反思"", ""智伯的覆亡讨论""]","""V_851""","""1.3.1"""
"""C_584313""","[""第二课 战国前期的政治"", ""魏文侯治国"", ""魏文侯治国""]","""V_857""","""2.1.1"""
"""C_584313""","[""第二课 战国前期的政治"", ""吴起的悲剧"", ""吴起的悲剧""]","""V_859""","""2.2.1"""
…,…,…,…
"""C_2329163""","[""第四章 红色之旅实践写作"", ""4.15 论文写作规范"", ""4.15 论文写作规范""]","""V_8630133""","""3.14"""
"""C_2329163""","[""第四章 红色之旅实践写作"", ""4.16 求职简历"", ""4.16 求职简历""]","""V_8630134""","""3.15"""
"""C_2329163""","[""第四章 红色之旅实践写作"", ""4.17 实践—演讲稿"", ""4.17 实践—演讲稿""]","""V_8630135""","""3.16"""


In [11]:
course_video = course_video.rename({
    "id": "course_id",
    "titles": "titles",
    "resource_id": "video_id",
    "chapter": "chapter",
})
course_exercise = course_exercise.rename({
    "id": "course_id",
    "titles": "titles",
    "resource_id": "exercise_id",
    "chapter": "chapter",
})

In [12]:
user_course = pl_read_file("/kaggle/input/preprocessing-user-json/user-course.csv")

In [13]:
common_courses = course_df.filter(pl.col('id').is_in(user_course['course_id']))

common_courses.describe()

statistic,id,name,field,prerequisites,about,resource
str,str,str,f64,str,str,f64
"""count""","""3781""","""3781""",3781.0,"""3779""","""3779""",3781.0
"""null_count""","""0""","""0""",0.0,"""2""","""2""",0.0
"""mean""",null,null,null,null,null,null
"""std""",null,null,null,null,null,null
"""min""","""C_1017355""",""" Food Chemistry """,null,"""""","""""",null
"""25%""",null,null,null,null,null,null
"""50%""",null,null,null,null,null,null
"""75%""",null,null,null,null,null,null
"""max""","""C_956450""","""（疾风计划）面向对象程序设计（C++）""",null,"""高级语言程序设计、数据结构""","""（1）特色：课程资源建设，充分体现优质资源的共享。老师从繁杂…",null


In [14]:
!pip install googletrans==3.1.0a0
from googletrans import Translator
translator = Translator()

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=15a0eaf0d6200e8cb40b111c3d59aa7af05df8dd773427c51ae2f9ebd0625c6b
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperframe

In [15]:
import polars as pl
from tqdm import tqdm
from googletrans import Translator

translator = Translator()

def translate_str_column(df, col, batch_size=200):
    texts = df[col].to_list()  # Ensure conversion to list
    trans_texts = []

    for i in tqdm(range(0, len(texts), batch_size)):
        texts_batch = [text if text else "" for text in texts[i: i + batch_size]]  # Handle NaNs
        trans_batch = translator.translate(texts_batch, dest='en')

        # Ensure translation was successful
        trans_batch = [trans.text.lower() if trans else "" for trans in trans_batch]
        trans_texts.extend(trans_batch)
        time.sleep(1) 
    assert len(trans_texts) == len(texts)

    return df.with_columns(pl.Series(name=f"{col}_trans", values=trans_texts))

# Example usage:
course_df = translate_str_column(course_df, 'about')


100%|██████████| 19/19 [03:04<00:00,  9.68s/it]


In [16]:
course_df = translate_str_column(course_df, 'prerequisites')
course_df = translate_str_column(course_df, 'name')

100%|██████████| 19/19 [02:55<00:00,  9.26s/it]


In [17]:
course_df.write_ndjson("course.json")